# 生成器与迭代器

Python 内置的字符串、列表、元组和字典是序列，也就是在这些类型中实现有 `__getitem__()` 魔术方法。同时它们还是可迭代对象。定义有`__iter__`方法d的对象称为可迭代对象。Python 内置的集合是可迭代对象，但不是序列。对于可迭代对象来说，可以用`for`循环来遍历它们的每一个元素，也称之为迭代，这也是可迭代对象的来由。

在Python中，万物皆对象，这也导致有些Python程序员不知道Python 的函数式编程。这里必须向大家道歉，要知道，函数式编程可以写出更为精确和高效的代码。

以函数式编程时，实现的函数最好是纯函数式函数。有点绕口，主要意思就是，一个函数接收参数然后输出结果，不会保留任何状态。

## 可迭代对象（Iterable）

通过检查是否定义 `__iter__` 方法来判断对象是否是可迭代对象。使用内置函数`hasattr()`可以检查检查字符串、列表、元组、字典、集合等对象：

通过检查是否定义了`__iter__`方法来判断可迭代对象。使用内置函数`hasattr()`可以检查检查字符串、列表、元组、字典、集合等对象是否是可迭代对象

In [5]:
for dtype in [str, list, tuple, dict, set]:
    print(str(dtype), hasattr(dtype(), '__iter__'))

<class 'str'> True
<class 'list'> True
<class 'tuple'> True
<class 'dict'> True
<class 'set'> True


显然整数、复数等数据类型不是可迭代对象：

In [6]:
for dtype in [int, bool, float, complex]:
    print(str(dtype), hasattr(dtype(), '__iter__'))

<class 'int'> False
<class 'bool'> False
<class 'float'> False
<class 'complex'> False


## 迭代器（Iterator）

迭代器(Iterator)是一种可以用`for`循环进行遍历的对象，此外还能使用内置函数`next()`依次返回元素。访问迭代器对象的元素只能从第一个元素，直到访问完所有元素，最后引起`StopIteration`异常。迭代器只能往后遍历不能回溯。通过检查是否定义`__next__`方法来判断迭代器对象。

In [8]:
for dtype in [str, list, tuple, dict, set]:
    print(str(dtype), hasattr(dtype(), '__next__'))

<class 'str'> False
<class 'list'> False
<class 'tuple'> False
<class 'dict'> False
<class 'set'> False


使用内置函数 `iter()` 可以把可迭代对象转化为迭代器:

In [16]:
xs = (1, 2, 3)
ixs1 = iter(xs)
ixs2 = iter([1, 2, 3])
print(type(ixs1), type(ixs2))
print(hasattr(ixs1, '__next__'), hasattr(ixs2, '__next__'))

<class 'tuple_iterator'> <class 'list_iterator'>
True True


下面就可以使用 Python 内置的`next()`来访问其中的元素，直到引起`StopIteration`异常：

In [18]:
print(next(ixs1))
print(next(ixs1))
print(next(ixs1))
print(next(ixs1))

StopIteration: 

In [19]:
hasattr(ixs2, '__iter__')

True

迭代器实现有`__iter__`方法，可以使用`for`语句来进行迭代循环：

In [21]:
for x in ixs2:
    print(x)

实际上，在实现对象的`__iter__()`方法时，就是返回一个迭代器。

## 生成器（generator）

### 有限的序列

软件是运行在计算机硬件上，超出硬件的限制就会出现错误，比较常见的如内存溢出错误。假如要创建一个大的列表，就需要创建很多个对象，自然会耗费大量的内存。如果超出内存限制就会出现内存溢出异常。

在Linux系统，使用`ulimit`命令把内存限制到256M。
```sh
ulimit -v 262144
```

然后启动Python程序，在提示符下运行如下语句：
```python
>>> alist = list(range(1024*1024))
>>> alist = list(range(1024*1024*1024))
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
MemoryError
```

这证明，限制内存后，是无法创建1024M个对象。如果在Python或Jupyter Notebook中试图分配超过物理内存大小，进程会被直接杀掉。

In [ ]:
# Don't run the statement. The kernal will be killed.
# alist = list(range(1024*1024*1024*8))

### 无限的生成器

在实际工作场景中，有时会需要得到非常众多的对象，然而并不需要一次性全部获得。Python 提供了生成器的方法，来依次得到不同对象。生成器（generator）是一个对象，通过调用内置函数`next()`可以依次返回不同值。可以使用定义函数的方法来创建一个生成器，要做的工作仅仅是使用`yield`语句：

In [22]:
def mygenerator():
    print('generator 1')
    yield 1
    print('generator 2')
    yield 3.1415
    print('generator 3')
    yield 'string'

In [3]:
print(type(mygenerator))

<class 'function'>
<function mygenerator at 0x7fa60829eea0>


在上面语句定义了一个函数`mygenerator`，可以看出就是函数对象，与其它对象没啥差别。不过该函数的返回结果则有所不同：

In [23]:
gen = mygenerator()

print(type(gen), gen)

<class 'generator'> <generator object mygenerator at 0x000001EB5A179620>


当 Python 检测到函数体中使用了 `yield` 语句,就将该函数标记为生成器函数。从上可知，函数返回的就是生成器对象。使用自省函数`dir()`检查生成器对象：

In [24]:
print(dir(gen))

['__class__', '__del__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__name__', '__ne__', '__new__', '__next__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'close', 'gi_code', 'gi_frame', 'gi_running', 'gi_yieldfrom', 'send', 'throw']


生成器对象包含了`__iter__`和`__next__`方法，也就是说生成器是迭代器，可以像迭代器那样进行迭代操作。下面使用内置函数`next()`来迭代生成器元素，直到最后引起 `StopIteration` 异常：

In [27]:
print(next(gen))
print(next(gen))
print(next(gen))
print(next(gen))

StopIteration: 

从运行结果可知，当函数执行到 `yield` 语句时，就会像遇到 `return` 那样返回表达式的结果。但有一个显著不同时，Python 解释器会保存当前运行位置，等下一次调用函数 `next()` 时，继续往下运行。

使用 Python 标准库 `inspect` 可以来检查函数是否是生成器函数：

In [28]:
import inspect

print(inspect.isfunction(mygenerator))
print(inspect.isgeneratorfunction(mygenerator))

True
True


### 简单应用

有了生成器，就可以应对和操作大规模数据集。回到刚开始的需求，动态创建一个大规模字符串序列。

In [10]:
def srange(n):
    i = 0
    while i < n:
        yield str(i)
        i += 1

In [11]:
for s in srange(1024*1024*8):
    if s == '8000000':
        print(s)

8000000


生成器通过在运行时生成值而非一次性生成所有值，大大减少了内存了消耗，使得可以处理大规模数据集和循环。从某种程度上讲，使用时间换空间。

### 内置的生成器

在前面的列表推导式章节中，提到“元组推导式”实际上是生成器：

In [29]:
xs = (i*i for i in range(3))
print(type(xs), xs)

<class 'generator'> <generator object <genexpr> at 0x000001EB5A218048>


Python 内置函数 `range()` 的语法是
```
range(stop) -> range object
range(start, stop[, step]) -> range object
``` 

`range()`返回一个`range`对象，与生成器函数功能有些类似，但它不是迭代器，即没有实现`__next__()`方法，也不能使用函数`next()`：

In [33]:
xs = range(3)
ys = range(1, 300, 100)
print(type(xs), type(ys))
print(hasattr(xs, '__next__'), hasattr(ys, '__next__'))

<class 'range'> <class 'range'>
False False


In [36]:
next(xs)

TypeError: 'range' object is not an iterator

但 `range` 对象定义了`__getitem__()`与`__iter__()`方法，是个可迭代对象，可以使用使用`for`循环：

In [41]:
print(hasattr(xs, '__getitem__'), hasattr(ys, '__getitem__'))
print(hasattr(xs, '__iter__'), hasattr(ys, '__iter__'))

for y in ys:
    print(y)

True True
True True
1
101
201


## 小结

Python解释器在迭代一个对象时，会自动调用`iter(x)`。其执行过程为：
1. 首先会查看该对象是否实现了`__iter__()`方法。如果实现则调用该方法，获取一个迭代器。
2. 如果没有实现`__iter__()`方法，会检查是否实现`__getitem()`方法，如实现则创建一个迭代器，从索引 0 开始获取元素。
3. 如果都失败，则抛出类型错误（TypeError），提示对象不是可迭代对象。

由上可知，可迭代（iterable）对象是一种能够迭代循环的对象。

序列对象是定义了`__getitem__()`方法的对象，可以通过整数索引来快捷访问其元素。序列对象是可迭代对象。

迭代器是定义有`__next__()`方法的，可以依次遍历元素但不能回溯。迭代器是可迭代对象。

生成器是一种特殊的迭代器。生成器也是可迭代对象。